# 本文档用于转换矢量数据投影,形成矢量数据数据库

数据准备
1. 统一CRS:`EPSG:32651`
2. 矢量数据
  1. 上海主城区面要素(`sh_main.shp`)
  2. 上海主城区乡镇面要素(`sh_main_town.shp`)

处理流程
1. 市中心面转换成5m乘5m格网
2. 市中心格网转换成点
3. 计算每个网格的供给与需求
   1. 供给的区域是所有包含蓝绿空间的网格
      1. 计算每个网格包含的蓝绿空间面积比重
         1. 
      2. 计算每个网格中ndvi的均值
      3. 计算每个网格中蓝绿空间的类型(如果出现一个网格中存在不同的蓝绿空间类型,则取面积最大的那个)
   2. 需求的区域是所有包含居民区building的网格
      1. 计算每个网格中的人口数量
      2. 计算每个网格中的外来人口数量
      3. 计算每个网格中的男性人口数量
      4. 计算每个网格中的女性人口数量
      5. 计算每个网格中三个年龄阶段的人口数量
      6. 计算每个网格中少数民族的数量
      7. 计算每个网格中的房价均值
      8. 以每个网格点为中心,绘制15分钟步行缓冲区
      9. 统计15分钟步行缓冲区内的供给总量
      10. 以该总量作为该网格点收到的供给总量
4. 计算供需匹配

In [1]:
# 初始化依赖包与qgis
import os
import numpy as np
import pandas as pd


import geopandas as gpd
import matplotlib.pyplot as plt

# import pyqgis
from qgis.core import *

# Supply path to qgis install location
QgsApplication.setPrefixPath('/Applications/QGIS-LTR.app/Contents/MacOS', True)

# Create a reference to the QgsApplication.  Setting the
# second argument to False disables the GUI.
qgs = QgsApplication([], False)

# Load providers
qgs.initQgis()

# Write your code here to load some layers, use processing
# algorithms, etc.

# Finally, exitQgis() is called to remove the
# provider and layer registries from memory


from qgis.analysis import QgsNativeAlgorithms
import processing
from processing.core.Processing import Processing
Processing.initialize()
QgsApplication.processingRegistry().addProvider(QgsNativeAlgorithms())
qgs.exitQgis()

Logged warning: Duplicate provider native registered


In [2]:
def getAttributeTable(vlayer):
    """ get attribute table of a vector layer.
    
    Args:
        vlayer : (QgsVectorLayer instance). vector layer

    returns:
         pandas DataFrame: attribute table
    """
    attribute_dict={}
    
    for field in vlayer.fields(): # 初始化各要素属性为空列表
        attribute_dict[field.name()] = []
        
    for feature in vlayer.getFeatures(): # 遍历矢量图层中个要素
        for field in vlayer.fields(): # 遍历各属性字段
            attribute_dict[field.name()].append(feature[field.name()]) # 添加属性字段值进入字典
            
            
    return pd.DataFrame(attribute_dict) #返回DataFrame

In [13]:
input_path = '/Users/oo/Desktop/Projects/esri-ces-contest/Codes/input/sh_main'
# pd.set_option('display.max_columns', None)
sh_main_green = QgsVectorLayer(r'{}/sh_main_green_fixed.shp'.format(input_path))
sh_main_ndvi_raster = QgsRasterLayer(r'{}/sh_main_ndvi.tif'.format(input_path))


In [14]:
temp_path = '/Users/oo/Desktop/Projects/esri-ces-contest/Codes/temp'
clip_ndvi = {'INPUT':sh_main_ndvi_raster, 
                      'MASK':sh_main_green, 
                      'OUTPUT': '{}/rsh_main_green_ndvi.tif'.format(temp_path)}

processing.run('gdal:cliprasterbymasklayer', clip_ndvi)

{'OUTPUT': '/Users/oo/Desktop/Projects/esri-ces-contest/Codes/temp/rsh_main_green_ndvi.tif'}